<a href="https://colab.research.google.com/github/Kaiziferr/NLP_Workshop/blob/master/seq2seq/w0_traductor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import keras

# Descarga los datos

In [2]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

# Configuration

In [3]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = "fra.txt"

# Prepara los datos

In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [5]:
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

In [6]:
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text, _ = line.split('\t')
  target_text = "\t" + target_text + "\n"
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
      if char not in input_characters:
          input_characters.add(char)
  for char in target_text:
      if char not in target_characters:
          target_characters.add(char)
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [7]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [8]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32'
)

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [9]:
# samples, orden, columnas
encoder_input_data.shape

(10000, 15, 71)

In [10]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.0
  encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] = 1.0
    if t > 0:
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
  decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
  decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Model

In [11]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [12]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [13]:
model.fit([encoder_input_data, decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2,)

Epoch 1/100
125/125 [==============================] - 14s 46ms/step - loss: 1.1377 - accuracy: 0.7361 - val_loss: 1.0103 - val_accuracy: 0.7200
Epoch 2/100
125/125 [==============================] - 4s 34ms/step - loss: 0.8124 - accuracy: 0.7798 - val_loss: 0.9067 - val_accuracy: 0.7688
Epoch 3/100
125/125 [==============================] - 4s 34ms/step - loss: 0.6480 - accuracy: 0.8181 - val_loss: 0.6888 - val_accuracy: 0.8036
Epoch 4/100
125/125 [==============================] - 4s 34ms/step - loss: 0.5650 - accuracy: 0.8356 - val_loss: 0.6236 - val_accuracy: 0.8145
Epoch 5/100
125/125 [==============================] - 4s 35ms/step - loss: 0.5198 - accuracy: 0.8479 - val_loss: 0.5783 - val_accuracy: 0.8326
Epoch 6/100
125/125 [==============================] - 4s 35ms/step - loss: 0.4855 - accuracy: 0.8567 - val_loss: 0.5541 - val_accuracy: 0.8388
Epoch 7/100
125/125 [==============================] - 4s 34ms/step - loss: 0.4576 - accuracy: 0.8645 - val_loss: 0.5370 - val_accuracy

In [20]:
#encoder_inputs = model.input[0] 
#encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  

print(state_h_enc, state_c_enc)

#encoder_states = [state_h_enc, state_c_enc]
#encoder_model = keras.Model(encoder_inputs, encoder_states)

"""decoder_inputs = model.input[1]  
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)
"""

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):

    states_value = encoder_model.predict(input_seq)


    target_seq = np.zeros((1, 1, num_decoder_tokens))

    target_seq[0, 0, target_token_index["\t"]] = 1.0


    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)


        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

   
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

 
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

      
        states_value = [h, c]
    return decoded_sentence

KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm/PartitionedCall:2', description="created by layer 'lstm'") KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm/PartitionedCall:3', description="created by layer 'lstm'")


In [23]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.